# A/B Testing the Udacity Website

Assignment descriptions: https://www.unifyingdatascience.org/html/exercises/exercise_abtesting.html <br>
Experiment discussed in this notebook (licensed under Apache 2.0): https://www.kaggle.com/tammyrotem/ab-tests-with-python/data

## Import the Data

In [5]:
import numpy as np
import pandas as pd
from scipy import stats

### Exercise 1

In [6]:
# treatment
daT = pd.read_csv("experiment_data.txt", sep = ",", header = 0)
daT.head()

,Date,Pageviews,Clicks,Enrollments,Payments
0,"Sat, Oct 11",7716,686,105.0,34.0
1,"Sun, Oct 12",9288,785,116.0,91.0
2,"Mon, Oct 13",10480,884,145.0,79.0
3,"Tue, Oct 14",9867,827,138.0,92.0
4,"Wed, Oct 15",9793,832,140.0,94.0


In [7]:
# control
daC = pd.read_csv("control_data.txt", sep = ",", header = 0)
daC.head()

,Date,Pageviews,Clicks,Enrollments,Payments
0,"Sat, Oct 11",7723,687,134.0,70.0
1,"Sun, Oct 12",9102,779,147.0,70.0
2,"Mon, Oct 13",10511,909,167.0,95.0
3,"Tue, Oct 14",9871,836,156.0,105.0
4,"Wed, Oct 15",10014,837,163.0,64.0


In [4]:
daC.shape

(37, 5)

### Exercise 2

Each row records the user's browsing history and course registration status on each date.

## Pick your measures

### Exercise 3

In [8]:
daT['Treatment'] = 1
daC['Treatment'] = 0

da = pd.concat([daT, daC])
# Convert Date
da['Date'] = pd.to_datetime(da['Date'], format = '%a, %b %d') \
                .apply(lambda x: x.replace(year = 2017))
da.sort_values(['Date', 'Treatment'], inplace = True)
da.head()

,Date,Pageviews,Clicks,Enrollments,Payments,Treatment
0,2017-10-11,7723,687,134.0,70.0,0
0,2017-10-11,7716,686,105.0,34.0,1
1,2017-10-12,9102,779,147.0,70.0,0
1,2017-10-12,9288,785,116.0,91.0,1
2,2017-10-13,10511,909,167.0,95.0,0


In [26]:
sum(da.groupby("Date")["Pageviews"].count() == 1)

0

0 means all records are pairs wised, one comes from the control group, and another comes from the treatment group.

### Exercise 4

In [27]:
da['NetConv'] = da['Payments'] / da['Clicks']
da['GrossConv'] = da['Enrollments'] / da['Clicks']
da.head()

,Date,Pageviews,Clicks,Enrollments,Payments,Treatment,NetConv,GrossConv
0,2017-10-11,7723,687,134.0,70.0,0,0.101892,0.195051
0,2017-10-11,7716,686,105.0,34.0,1,0.049563,0.153061
1,2017-10-12,9102,779,147.0,70.0,0,0.089859,0.188703
1,2017-10-12,9288,785,116.0,91.0,1,0.115924,0.147771
2,2017-10-13,10511,909,167.0,95.0,0,0.104510,0.183718


## Validating The Data

### Exercise 5

In [28]:
print("Average Pageviews of Treatment Group is {:.3f}".format(
np.mean(da[da['Treatment'] == 1]["Pageviews"])))
print("Average Pageviews of Control Group is {:.3f}".format(
np.mean(da[da['Treatment'] == 0]["Pageviews"])))

Average Pageviews of Treatment Group is 9315.135
Average Pageviews of Control Group is 9339.000


### Exercise 6

In [30]:
test_res = stats.ttest_ind(da[da['Treatment'] == 1]["Pageviews"],
da[da['Treatment'] == 0]["Pageviews"])
print("The p-value of Treatment t-test is {:.3f}".format(test_res.pvalue))

The p-value of Treatment t-test is 0.888


### Exercise 7

* Clicks
* Click-through-probability (CTP) of the Free Trial Button. The CTP is cauculated by $CPT = \frac{Pageviews}{Clicks}$

### Exercise 8

In [33]:
test_res = stats.ttest_ind(da[da['Treatment'] == 1]["Clicks"],
da[da['Treatment'] == 0]["Clicks"])
print("The p-value of Clicks t-test is {:.3f}".format(test_res.pvalue))

The p-value of Clicks t-test is 0.926


In [34]:
da["CPT"] = da["Pageviews"] / da["Clicks"]
da.head()

,Date,Pageviews,Clicks,Enrollments,Payments,Treatment,NetConv,GrossConv,CPT
0,2017-10-11,7723,687,134.0,70.0,0,0.101892,0.195051,11.241630
0,2017-10-11,7716,686,105.0,34.0,1,0.049563,0.153061,11.247813
1,2017-10-12,9102,779,147.0,70.0,0,0.089859,0.188703,11.684211
1,2017-10-12,9288,785,116.0,91.0,1,0.115924,0.147771,11.831847
2,2017-10-13,10511,909,167.0,95.0,0,0.104510,0.183718,11.563256


In [35]:
test_res = stats.ttest_ind(da[da['Treatment'] == 1]["CPT"],
da[da['Treatment'] == 0]["CPT"])
print("The p-value of CPT t-test is {:.3f}".format(test_res.pvalue))

The p-value of CPT t-test is 0.924


## Estimating the Effect of Experiment

### Exercise 9

In [36]:
# There are null data in the dateset, drop them
full_data = da.dropna()

test_res = stats.ttest_ind(full_data[full_data['Treatment'] == 1]["NetConv"],
full_data[full_data['Treatment'] == 0]["NetConv"])
print("The p-value of Net Conversion t-test is {:.3f}".format(test_res.pvalue))
test_res = stats.ttest_ind(full_data[full_data['Treatment'] == 1]["GrossConv"],
full_data[full_data['Treatment'] == 0]["GrossConv"])
print("The p-value of Gross Conversion t-test is {:.3f}".format(test_res.pvalue))

The p-value of Net Conversion t-test is 0.593
The p-value of Gross Conversion t-test is 0.131


### Exercise 10

In [42]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
smf.ols("GrossConv ~ Treatment", full_data).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              GrossConv   R-squared:                       0.051
Model:                            OLS   Adj. R-squared:                  0.030
Method:                 Least Squares   F-statistic:                     2.371
Date:                Tue, 09 Feb 2021   Prob (F-statistic):              0.131
Time:                        11:54:46   Log-Likelihood:                 77.613
No. Observations:                  46   AIC:                            -151.2
Df Residuals:                      44   BIC:                            -147.6
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.2204      0.010     23.084      0.000       0.201       0.240
Treatment     -0.0208      0.013     -1.540      0.131      -0.048       0.006
==============================================================================
Omnibus:                        6.181   Durbin-Watson:                   0.677
Prob(Omnibus):                  0.045   Jarque-Bera (JB):                6.094
Skew:                           0.850   Prob(JB):                       0.0475
Kurtosis:                       2.460   Cond. No.                         2.62
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [43]:
smf.ols("NetConv ~ Treatment", full_data).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                NetConv   R-squared:                       0.007
Model:                            OLS   Adj. R-squared:                 -0.016
Method:                 Least Squares   F-statistic:                    0.2903
Date:                Tue, 09 Feb 2021   Prob (F-statistic):              0.593
Time:                        11:54:49   Log-Likelihood:                 95.810
No. Observations:                  46   AIC:                            -187.6
Df Residuals:                      44   BIC:                            -184.0
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.1183      0.006     18.403      0.000       0.105       0.131
Treatment     -0.0049      0.009     -0.539      0.593      -0.023       0.013
==============================================================================
Omnibus:                        0.968   Durbin-Watson:                   1.165
Prob(Omnibus):                  0.616   Jarque-Bera (JB):                0.985
Skew:                           0.316   Prob(JB):                        0.611
Kurtosis:                       2.662   Cond. No.                         2.62
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Exercise 11

In [44]:
smf.ols("NetConv ~ Treatment + Date", full_data).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                NetConv   R-squared:                       0.743
Model:                            OLS   Adj. R-squared:                  0.475
Method:                 Least Squares   F-statistic:                     2.770
Date:                Tue, 09 Feb 2021   Prob (F-statistic):            0.00991
Time:                        11:55:01   Log-Likelihood:                 126.94
No. Observations:                  46   AIC:                            -205.9
Df Residuals:                      22   BIC:                            -162.0
Df Model:                          23                                         
Covariance Type:            nonrobust                                         
============================================================================================================
                                               coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------
Intercept                                    0.0782      0.016      4.885      0.000       0.045       0.111
Date[T.Timestamp('2017-10-12 00:00:00')]     0.0272      0.022      1.226      0.233      -0.019       0.073
Date[T.Timestamp('2017-10-13 00:00:00')]     0.0212      0.022      0.957      0.349      -0.025       0.067
Date[T.Timestamp('2017-10-14 00:00:00')]     0.0427      0.022      1.927      0.067      -0.003       0.089
Date[T.Timestamp('2017-10-15 00:00:00')]     0.0190      0.022      0.857      0.401      -0.027       0.065
Date[T.Timestamp('2017-10-16 00:00:00')]     0.0128      0.022      0.578      0.569      -0.033       0.059
Date[T.Timestamp('2017-10-17 00:00:00')]     0.0033      0.022      0.148      0.884      -0.043       0.049
Date[T.Timestamp('2017-10-18 00:00:00')]     0.0272      0.022      1.228      0.233      -0.019       0.073
Date[T.Timestamp('2017-10-19 00:00:00')]     0.0229      0.022      1.035      0.312      -0.023       0.069
Date[T.Timestamp('2017-10-20 00:00:00')]     0.0376      0.022      1.696      0.104      -0.008       0.084
Date[T.Timestamp('2017-10-21 00:00:00')]     0.0259      0.022      1.170      0.255      -0.020       0.072
Date[T.Timestamp('2017-10-22 00:00:00')]     0.0229      0.022      1.032      0.313      -0.023       0.069
Date[T.Timestamp('2017-10-23 00:00:00')]     0.0193      0.022      0.873      0.392      -0.027       0.065
Date[T.Timestamp('2017-10-24 00:00:00')]     0.0823      0.022      3.717      0.001       0.036       0.128
Date[T.Timestamp('2017-10-25 00:00:00')]     0.0774      0.022      3.495      0.002       0.031       0.123
Date[T.Timestamp('2017-10-26 00:00:00')]     0.0706      0.022      3.186      0.004       0.025       0.117
Date[T.Timestamp('2017-10-27 00:00:00')]     0.0831      0.022      3.752      0.001       0.037       0.129
Date[T.Timestamp('2017-10-28 00:00:00')]     0.0676      0.022      3.052      0.006       0.022       0.114
Date[T.Timestamp('2017-10-29 00:00:00')]     0.0485      0.022      2.188      0.040       0.003       0.094
Date[T.Timestamp('2017-10-30 00:00:00')]     0.0214      0.022      0.965      0.345      -0.025       0.067
Date[T.Timestamp('2017-10-31 00:00:00')]     0.0810      0.022      3.655      0.001       0.035       0.127
Date[T.Timestamp('2017-11-01 00:00:00')]     0.0645      0.022      2.911      0.008       0.019       0.110
Date[T.Timestamp('2017-11-02 00:00:00')]     0.0437      0.022      1.974      0.061      -0.002       0.090
Treatment                                   -0.0049      0.007     -0.750      0.461      -0.018       0.009
==============================================================================
Omnibus:                        4.114   Durbin-Watson:       

### Exercise 12

Not to adopt the change.

### Exercise 13

In [40]:
import calendar
da["DoW"] = [calendar.day_name[x.weekday() ] for x in da["Date"] ]
da.head()

,Date,Pageviews,Clicks,Enrollments,Payments,Treatment,NetConv,GrossConv,CPT,DoW
0,2017-10-11,7723,687,134.0,70.0,0,0.101892,0.195051,11.241630,Wednesday
0,2017-10-11,7716,686,105.0,34.0,1,0.049563,0.153061,11.247813,Wednesday
1,2017-10-12,9102,779,147.0,70.0,0,0.089859,0.188703,11.684211,Thursday
1,2017-10-12,9288,785,116.0,91.0,1,0.115924,0.147771,11.831847,Thursday
2,2017-10-13,10511,909,167.0,95.0,0,0.104510,0.183718,11.563256,Friday
